In [4]:
import logging
import sys
from dotenv import load_dotenv
from llama_index.core import StorageContext, get_response_synthesizer, load_index_from_storage
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings
import pickle
import os

load_dotenv()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


In [ ]:
# Download query engine from Azure Blob Storage Container
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# Your storage account connection string
connection_string = "DefaultEndpointsProtocol=https;AccountName=finessetestblobstorage;AccountKey=;EndpointSuffix=core.windows.net"

# The name of your container
container_name = "llamaindex-v1"

# The name of the virtual folder you want to list files from
folder_name = "index"

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get the container client
container_client = blob_service_client.get_container_client(container_name)

# List all blobs in the specified folder
blobs_list = container_client.list_blobs(name_starts_with=folder_name)

# List all blobs in the container (at the root)
blobs_list = container_client.list_blobs()

for blob in blobs_list:
    print("Blob name: " + blob.name)
    blob_name = blob.name
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    # Download the blob to a local file
    download_file_path = "./index/" + blob_name
    with open(download_file_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())




In [6]:
llm = AzureOpenAI(
    model="gpt-4",
    deployment_name="ailab-llm",
    api_key=os.getenv("API_KEY"),
    azure_endpoint=os.getenv("AZURE_ENDPOINT"),
    api_version=os.getenv("API_VERSION"),
)

embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="ada",
    api_key=os.getenv("API_KEY"),
    azure_endpoint=os.getenv("AZURE_ENDPOINT"),
    api_version=os.getenv("API_VERSION"),
)

Settings.llm = llm
Settings.embed_model = embed_model

In [2]:
print("rebuild storage context")
storage_context = StorageContext.from_defaults(persist_dir="../index")
with open('storage_context.pkl', 'wb') as file:
    pickle.dump(storage_context, file)

rebuild storage context


In [7]:

# print("load storage_context")
# with open('storage_context.pkl', 'rb') as file:
#     storage_context = pickle.load(file)

print("load index")
index = load_index_from_storage(storage_context)

print("configure retriever for debugging and retrieving metadata")
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=15,
)

print("configure response synthesizer")
response_synthesizer = get_response_synthesizer()

print("assemble query engine")
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)


load storage_context
load index
INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
configure retriever for debugging and retrieving metadata
configure response synthesizer
assemble query engine


In [8]:
# Example of query execute
response = query_engine.query("How do I import a cat from France to Canada?")
print(response)

INFO:httpx:HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ada/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ada/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ada/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ada/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ailab-llm/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://azure-open-ai-common-infra.openai.azure.com//openai/deployments/ailab-llm/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https

In [9]:
# get top k result into a list, in order of match score
top_k_result = []
for i in range(15): # arbitrary 15 because similarity_top_k=15 in this example
    top_k_result.append(response.source_nodes[i])
    
# get content
response.source_nodes[0].get_content()
# get embedding
response.source_nodes[0].embedding
# get score
response.source_nodes[0].get_score()
# get customized metadata. In this example, this retrieves chunk_id
response.source_nodes[0].metadata

{'id_': '558835ec-1285-428c-bc83-89c30128f68a'}